### Libraries

In [ ]:
from urllib.request import Request, urlopen
import requests
import json
import csv
#TODO: optimize imports

### quick authorization and setup:

In [ ]:
_client_id = ''
_secret_id = ''

print("https://trakt.tv/oauth/authorize?response_type=code&client_id={0}&redirect_uri=urn:ietf:wg:oauth:2.0:oob".format(_client_id))
_pincode = str(input('Input PIN: '))  
url = 'https://api.trakt.tv' + '/oauth/token'

values = {
        "code": _pincode,
        "client_id": _client_id,
        "client_secret": _secret_id,
        "redirect_uri": "urn:ietf:wg:oauth:2.0:oob",
        "grant_type": "authorization_code"
        }

request = requests.post(url, data=values)
response = request.json()
Authorization = 'Bearer ' + response["access_token"]

headers = {
  'Content-Type': 'application/json',
  'Authorization': Authorization,
  'trakt-api-version': '2',
  'trakt-api-key': _client_id
}


### info about a show:

In [ ]:
def getLastWatchedEpisode(showTraktID):
    
    link_to_show = "https://api.trakt.tv/shows/" + str(showTraktID) + "/progress/watched"
    request = Request(link_to_show, headers=headers)

    response_body = urlopen(request).read()
    my_json = response_body.decode('utf8')

    data = json.loads(my_json)
    # s = json.dumps(data, indent=4, sort_keys=True)
    # print(s)
#     print(data['last_episode']['season'], " ", data['last_episode']['number'])
    
    return data

### info about the last aired episode for a show

In [ ]:
def getLastAiredEpisode(showTraktID):
    
    link_to_show = "https://api.trakt.tv/shows/" + str(showTraktID) + "/last_episode"
    request = Request(link_to_show, headers=headers)

    response_body = urlopen(request).read()
    my_json = response_body.decode('utf8')

    data = json.loads(my_json)
    
    return data

### Get a list of episodes(to be aired) for a week:

In [ ]:
def getEpisodesNextWeek(date="2020-03-16", period=7):
    """
        start date in format: yyyy-mm-dd 
        /
        num of days in format: int
    """

    link_to_req = "https://api.trakt.tv/calendars/my/shows/" + str(date) + "/"+ str(period)

    request = Request(link_to_req, headers=headers)
    response_body = urlopen(request).read()

    my_json = response_body.decode('utf8')

    data = json.loads(my_json)
    
    # s = json.dumps(data, indent=4, sort_keys=True)
    # print(s)
    return data
#     for x in data:
#         print(x['show'])
        
    #TODO: make a proper return

### get list of all watching shows:

In [ ]:
def getAllShowsForUser(username = 'darknessest'):
    
    link_all_shows = 'https://api.trakt.tv/users/' + username + '/watched/' + 'shows' + '?extended=noseasons'
    request = Request(link_all_shows, headers=headers)

    response_body = urlopen(request).read()
    my_json = response_body.decode('utf8')

    data = json.loads(my_json)
    print(len(data))
    
#     for x in data:
#         print(x['show']['title'], " ", x['show']['ids']['trakt'])
    
    return data.copy()
        
    #TODO: make a proper return

### Constructing table with shows information:

In [ ]:
# shit = getEpisodesNextWeek()

In [ ]:
# for x in shit:
#     print(x['episode'])

In [ ]:
import pandas as pd

showsProgressTable = pd.DataFrame(columns=['name', 'traktID', 'last watched season', 
                                        'last watched episode', 'next episode air time',
                                        'new season', 'new episode'])

In [ ]:
all_shows_data = getAllShowsForUser()

In [ ]:
showsProgressTable['name'] = [x['show']['title'] for x in all_shows_data]
showsProgressTable['traktID'] = [x['show']['ids']['trakt'] for x in all_shows_data]

In [ ]:
print(showsProgressTable.head())
# temp_row = showsProgressTable.loc[showsProgressTable['traktID'] == 99718]

# print(temp_row)

### Fill in info about last shows progress

In [ ]:
import time
from random import randint

for i in range(24, len(showsProgressTable.index)):
#     print(row['name'])
    print("working on ", showsProgressTable.loc[i].at['name'], " ", i, " of ", len(showsProgressTable.index)-1 )
    ID = showsProgressTable.loc[i].at['traktID']

    info = getLastWatchedEpisode(ID)

    # set values
    showsProgressTable.at[i, 'last watched season'] = info['last_episode']['season']
    showsProgressTable.at[i, 'last watched episode'] = info['last_episode']['number']
    time.sleep(randint(5, 10))
# check
if showsProgressTable['last watched season'].notna().sum() == len(showsProgressTable.index) and \
    showsProgressTable['last watched episode'].notna().sum() == len(showsProgressTable.index):
        print('last watched info was downloaded completely')

### Fill in info about release dates (for a next week):

In [ ]:
table = getEpisodesNextWeek()

In [ ]:
for x in table:
    showsProgressTable.loc[showsProgressTable['traktID'] == x['show']['ids']['trakt'], ['next episode air time']] = x['first_aired']
    showsProgressTable.loc[showsProgressTable['traktID'] == x['show']['ids']['trakt'], ['new season']] = x['episode']['season']
    showsProgressTable.loc[showsProgressTable['traktID'] == x['show']['ids']['trakt'], ['new episode']] = x['episode']['number']
    

In [ ]:
print(showsProgressTable)

### convert data types:

In [ ]:
showsProgressTable['next episode air time'] = pd.to_datetime(showsProgressTable['next episode air time'])

showsProgressTable = showsProgressTable.astype({'name':'str',
                                                'traktID':'int8',
                                                'last watched season':'int8',
                                                'last watched episode':'int8',
#                                                 'new season':'int',
#                                                 'new episode':'int'
                                               })

### save table to a file:

In [ ]:
# save
showsProgressTable.to_hdf('pd_progress.h5', key='df', mode='w')


### Fill in the rest of the empty cells:

In [ ]:
for i in range(0, len(showsProgressTable.index)):
    if pd.isna(showsProgressTable.at[i, 'new season']) == True:
        print("working on", showsProgressTable.loc[i].at['name'], " ", i, " of ", len(showsProgressTable.index)-1)
        ID = showsProgressTable.loc[i].at['traktID']
        info = getLastAiredEpisode(ID)

        # set values
        showsProgressTable.at[i, 'new season'] = info['season']
        showsProgressTable.at[i, 'new episode'] = info['number']
        time.sleep(randint(5, 10))
        
# check
if showsProgressTable['new season'].notna().sum() == len(showsProgressTable.index) and \
    showsProgressTable['new episode'].notna().sum() == len(showsProgressTable.index):
        print('new episode info was downloaded completely')

In [ ]:
print(showsProgressTable.info())

In [ ]:
tab_c = showsProgressTable.copy()

tab_c['next episode air time'] = pd.to_datetime(tab_c['next episode air time'])

tab_c = tab_c.astype({'name':'str',
                      'traktID':'uint64',
                      'last watched season':'uint8',
                      'last watched episode':'uint8',
                      'new season':'uint8',
                      'new episode':'uint8'
                                               })

In [ ]:
tab_c.head()

In [ ]:
tab_c.to_hdf('pd_progress.h5', key='df', mode='w')